In [17]:
from py2cytoscape import cyrest
from statistics import *
from math import *
from numpy.random import randint

In [33]:
#Get data from current network
cy = cyrest.cyclient()
data = {}

#Rounding

raw_data = cy.node.get_properties(network='CURRENT', nodeList='all', propertyList='Y Location, Height')
heights = []
for item in raw_data:
    for prop in item['visualProperties']:
        if prop['visualProperty'] == 'NODE_HEIGHT':
            heights.append(prop['value'])
min_height = min(heights)
magnitude = floor(log10(min_height / 100))
                  
if magnitude <= 0:
    for entry in raw_data:
        data[str(entry['SUID'])] = round(
            [
                prop['value'] 
                for prop in entry['visualProperties'] 
                if prop['visualProperty'] == 'NODE_Y_LOCATION'
            ][0],
            -magnitude
        )
else:
    for entry in raw_data:
        data[str(entry['SUID'])] = round(
            [
                prop['value'] 
                for prop in entry['visualProperties'] 
                if prop['visualProperty'] == 'NODE_Y_LOCATION'
            ][0] 
            / 10 ** magnitude
        ) * (10 ** magnitude)
'''

raw_data = cy.node.get_properties(network='CURRENT', nodeList='all', propertyList='Y Location')
for entry in raw_data:
    data[str(entry['SUID'])] = entry['visualProperties'][0]['value']
'''     
data

{'303': -422.4,
 '304': -323.6,
 '305': -570.2,
 '306': -428.3,
 '307': 163.4,
 '308': -60.5,
 '309': -568.3,
 '310': 43.7,
 '311': -185.5,
 '312': -315.8,
 '283': 286.4,
 '284': -323.6,
 '285': -323.6,
 '286': 294.2,
 '287': -183.6,
 '288': -562.4,
 '289': -321.6,
 '290': -175.8,
 '291': -568.3,
 '292': -422.4,
 '293': 41.7,
 '294': 169.2,
 '295': -60.5,
 '296': -418.6,
 '297': -56.6,
 '298': -564.4,
 '299': -576.0,
 '300': -416.6,
 '301': -416.6,
 '302': -420.5}

In [35]:
#Add 1/1000 of largest variance as pseudocount

clusters = [{'members': [item[0]], 'values': [item[1]]} for item in data.items()]
iterations = 0
variances = []

while len(clusters) > 1:
    iterations += 1
    #Find closest clusters
    smallest_diff = float('Infinity')
    i_coord = None
    j_coord = None
    for i in range(len(clusters)-1):
        for j in range(i+1, len(clusters)):
            diff = abs(mean(clusters[i]['values']) - mean(clusters[j]['values']))
            if diff < smallest_diff:
                smallest_diff = diff
                i_coord = i
                j_coord = j
    #Merge closest clusters
    clusters[i_coord]['members'] += clusters[j_coord]['members']
    clusters[i_coord]['values'] += clusters[j_coord]['values']
    del(clusters[j_coord])
    
    sum_of_variances_within = 0
    for cluster in clusters:
        if len(cluster['members']) > 1:
            sum_of_variances_within += variance(cluster['values'])

    variances.append(sum_of_variances_within)
    
pseudocount = max(variances) / 1000
print(pseudocount)
for var in variances:
    print(var)# + pseudocount)

71.09693688505747
0.0
0.0
0.0
0.0
0.0
0.0
1.2033333333333047
3.0083333333333155
4.211666666666764
5.211666666666764
7.211666666666764
9.211666666666764
10.545000000000098
15.615000000000094
20.16800000000001
27.251666666666644
44.07166666666655
54.47966666666658
84.89966666666666
109.51799999999992
119.35738095238091
131.65304761904758
2722.3529696969686
5833.400969696969
10927.240969696968
16705.619457792207
24152.475115546215
40156.81067290552
71096.93688505747
